In [ ]:
!pip uninstall -y imagebind
!pip install git+https://github.com/Hackathon-Hitchhiking/ImageBind.git

In [ ]:
!pip install torch catboost librosa torchaudio 

In [ ]:
!pip install decord

In [1]:
import torch
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import os
import tempfile
import subprocess
import librosa
from imagebind import model
from imagebind.model import ModalityType 
from imagebind.utils import data
from loguru import logger
import decord

In [2]:
class PersonalityDataset(Dataset):
    def __init__(self, csv_file, path_to_video, imagebind_model, device):
        self.data = pd.read_csv(csv_file)
        self.path_to_video = path_to_video
        self.device = device

        imagebind_model.eval()
        imagebind_model.to(device)
        self.imagebind_model = imagebind_model

    def __len__(self):
        return len(self.data)

    def extract_audio_embedding(self, video_path):
        """
        Извлекает аудиовектор из видео.
        
        Параметры
        ----------
        video_path : str
            Путь к видеофайлу, из которого необходимо извлечь аудиодорожку.
        
        Возвращаемое значение
        ----------------------
        tuple
            Кортеж, содержащий:
            - audio : numpy.ndarray
                Массив, представляющий аудиоданные, загруженные из временного файла.
            - sr : int
                Частота дискретизации аудиоданных.
            - audio_embeddings : torch.Tensor
                Векторное представление аудиоданных, извлеченное моделью ImageBind.
        
        Описание
        ---------
        Данная функция использует библиотеку ffmpeg для извлечения аудиодорожки из
        видеофайла и сохранения её во временном WAV-файле. Затем с помощью библиотеки
        librosa аудиоданные загружаются и преобразуются в массив. После этого аудиоданные
        обрабатываются моделью ImageBind, чтобы получить векторное представление аудио.
        Функция возвращает массив аудиоданных, частоту дискретизации и среднее значение
        векторных представлений аудио.
        
        Исключения
        ----------
        - subprocess.CalledProcessError
            Генерируется, если выполнение команды ffmpeg завершилось с ошибкой.
        - FileNotFoundError
            Генерируется, если ffmpeg или необходимые библиотеки не установлены.
        """
        with tempfile.NamedTemporaryFile(suffix='.wav', delete=True) as temp_audio_file:
            temp_audio_path = temp_audio_file.name
            subprocess.run(
                ["ffmpeg", "-y", "-i", video_path, temp_audio_path, "-loglevel", "error"],
                check=True
            )
            audio, sr = librosa.load(temp_audio_path, sr=None)

            inputs = {ModalityType.AUDIO: data.load_and_transform_audio_data([temp_audio_path], self.device)}
            with torch.inference_mode():
                audio_embeddings = self.imagebind_model(inputs)[ModalityType.AUDIO].mean(dim=0)

            return audio, sr, audio_embeddings

    def extract_video_embedding(self, video_path):
        """
        Извлекает эмбеддинги видео, с логированием размеров каждого клипа.
        
        Параметры
        ----------
        video_path : str
            Путь к видеофайлу, для которого необходимо извлечь эмбеддинги.
        
        Возвращает
        -------
        torch.Tensor
            Тензор эмбеддинга видео со средним значением по всем клипам.
            Если клипы не были извлечены, возвращается тензор нулей с размерностью (1024,).
        
        Примечания
        --------
        - Данная функция использует модель ImageBind для обработки клипов видео и извлечения эмбеддингов.
        - Для повышения производительности используется torch.inference_mode().
        - Если видео успешно загружается и преобразуется, каждый клип обрабатывается отдельно, после чего
          их эмбеддинги усредняются.
        - Если клипов нет (например, видеофайл пуст или не удалось загрузить), возвращается тензор нулей.
        - Очищается кэш CUDA в конце для освобождения видеопамяти.
        
        Пример
        -------
        >>> video_embedding = extractor.extract_video_embedding('/path/to/video.mp4')
        >>> print(video_embedding.shape)
        torch.Size([1024])
        """
        embeddings_list = []
        with torch.inference_mode():
            video_clips = data.load_and_transform_video_data([video_path], self.device)
            if video_clips is not None:
                # video_clips shape: (num_clips, C, T, H, W)
                for i, video_clip in enumerate(video_clips):
                    # Add batch dimension if necessary
                    if video_clip.dim() == 4:
                        video_clip = video_clip.unsqueeze(0)
                    chunk_embeddings = self.imagebind_model({ModalityType.VISION: video_clip})
                    embeddings_list.append(chunk_embeddings[ModalityType.VISION].mean(dim=0))

        video_embedding = torch.stack(embeddings_list).mean(dim=0) if embeddings_list else torch.zeros(1024, device=self.device)
        torch.cuda.empty_cache()
        return video_embedding

    def extract_text_embedding(self, text):
        """
        Извлекает эмбеддинг для текста.
        
        Параметры
        ----------
        text : str
            Входной текст для извлечения эмбеддинга. Если строка пустая или не является строкой,
            будет использовано значение по умолчанию "<UNK>".
        
        Возвращает
        -------
        torch.Tensor
            Эмбеддинг текста, полученный с помощью модели ImageBind.
        
        Примечания
        ---------
        Функция использует метод `load_and_transform_text` для предварительной обработки текста,
        приводя его в формат, подходящий для обработки моделью. Затем эмбеддинг извлекается
        в режиме inference, что позволяет выполнять вычисления без сохранения промежуточных
        данных для обучения.
        
        Исключения
        ---------
        Проверка типа входных данных выполняется с целью обеспечения безопасности и предотвращения
        ошибок при передаче некорректного формата. Если переданный текст не соответствует
        ожидаемому типу, используется placeholder "<UNK>".
        """
        if not isinstance(text, str) or not text:
            text = "<UNK>"
        inputs = {ModalityType.TEXT: data.load_and_transform_text([text], self.device)}
        with torch.inference_mode():
            text_embedding = self.imagebind_model(inputs)[ModalityType.TEXT]
        return text_embedding

    def extract_keypoints(self, video_path, visualize=False):
        """
        Извлекает ключевые точки позы, лица и рук из видео и возвращает среднее значение этих ключевых точек.
        
        Параметры
        ----------
        video_path : str
            Путь к видеофайлу, из которого нужно извлечь ключевые точки.
        visualize : bool, optional
            Флаг, указывающий, нужно ли визуализировать ключевые точки на первом кадре видео (по умолчанию False).
        
        Возвращает
        -------
        np.ndarray
            Среднее значение ключевых точек по всем кадрам видео. Массив содержит координаты x, y, z и видимость для поз,
            а также координаты x, y, z для лица и рук. Если в видео отсутствуют ключевые точки, возвращается массив нулей.
        
        Описание
        --------
        Функция использует библиотеку Mediapipe для анализа видео и извлечения ключевых точек человеческого тела,
        включая позы, лицо и руки. Видео обрабатывается покадрово, и для каждого кадра вычисляются ключевые точки.
        Если ключевые точки отсутствуют в каком-либо кадре, заполняются нулевые значения.
        
        В случае, если флаг `visualize` установлен в True, функция отобразит ключевые точки на первом кадре видео
        с использованием matplotlib и OpenCV.
        
        Примечание
        ---------
        1. Функция использует Mediapipe Holistic для извлечения ключевых точек, включая улучшенные маркеры лица.
        2. Для работы требуется установленная библиотека OpenCV (cv2), matplotlib и numpy.
        
        Пример
        -------
        >>> keypoints = self.extract_keypoints("path/to/video.mp4", visualize=True)
        >>> print(keypoints)
        
        """
        mp_holistic = mp.solutions.holistic
        mp_drawing = mp.solutions.drawing_utils

        holistic = mp_holistic.Holistic(
            static_image_mode=False,
            model_complexity=2,
            enable_segmentation=False,
            refine_face_landmarks=True
        )

        cap = cv2.VideoCapture(video_path)
        keypoints_list = []

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Преобразование изображения в формат RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Получение результатов
            results = holistic.process(image)

            keypoints = []

            # Позы
            if results.pose_landmarks:
                for landmark in results.pose_landmarks.landmark:
                    keypoints.extend([landmark.x, landmark.y,
                                      landmark.z, landmark.visibility])
            else:
                keypoints.extend([0] * 33 * 4)

            # Лицо
            if results.face_landmarks:
                for landmark in results.face_landmarks.landmark:
                    keypoints.extend([landmark.x, landmark.y, landmark.z])
            else:
                keypoints.extend([0] * 468 * 3)

            # Левая рука
            if results.left_hand_landmarks:
                for landmark in results.left_hand_landmarks.landmark:
                    keypoints.extend([landmark.x, landmark.y, landmark.z])
            else:
                keypoints.extend([0] * 21 * 3)

            # Правая рука
            if results.right_hand_landmarks:
                for landmark in results.right_hand_landmarks.landmark:
                    keypoints.extend([landmark.x, landmark.y, landmark.z])
            else:
                keypoints.extend([0] * 21 * 3)

            keypoints_list.append(keypoints)

            if visualize and frame_count == 1:
                # Визуализация ключевых точек на первом кадре
                annotated_image = frame.copy()
                # Позы
                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        annotated_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                # Лицо
                if results.face_landmarks:
                    mp_drawing.draw_landmarks(
                        annotated_image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
                # Левая рука
                if results.left_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                # Правая рука
                if results.right_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                # Преобразование изображения в формат RGB для отображения
                annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
                # Отображение изображения в Jupyter Notebook
                plt.figure(figsize=(10, 10))
                plt.imshow(annotated_image)
                plt.axis('off')
                plt.title(f"Ключевые точки для видео: {os.path.basename(video_path)}")
                plt.show()

        cap.release()
        cv2.destroyAllWindows()
        holistic.close()

        # Усредняем ключевые точки по всем кадрам
        keypoints_array = np.array(keypoints_list)
        if keypoints_array.size == 0:
            num_keypoints = (33 * 4) + (468 * 3) + (21 * 3 * 2)
            keypoints_mean = np.zeros(num_keypoints)
        else:
            keypoints_mean = keypoints_array.mean(axis=0)

        return keypoints_mean

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        video_file = row['video_file']
        video_path = os.path.join(self.path_to_video, video_file)
        if not os.path.isfile(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")

        logger.debug(f"Processing video file: {video_path}")

        # Extract embeddings
        audio, sr, audio_embeddings = self.extract_audio_embedding(video_path)
        # video_embedding = self.extract_video_embedding(video_path)
        transcript_embeddings = self.extract_text_embedding(row['transcript'])

        key_points = self.extract_keypoints(video_path)

        # Prepare labels
        labels = {
            'extraversion': row['extraversion'],
            'neuroticism': row['neuroticism'],
            'agreeableness': row['agreeableness'],
            'conscientiousness': row['conscientiousness'],
            'openness': row['openness'],
            'interview': row['interview']
        }

        # Assemble all data into a dictionary
        sample = {
            'video_file': video_file,
            'audio': audio,
            'audio_embeddings': audio_embeddings,
            'sampling_rate': sr,
            'transcript': row['transcript'],
            "transcript_embeddings": transcript_embeddings,
            'labels': labels,
            'key_points': key_points,
        }

        return sample


In [3]:
imagebind_model = model.imagebind_huge(True)

/home/user1/environments/hack/lib/python3.10/site-packages/imagebind/model.py:513: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(".checkpoin

In [4]:
train_data = PersonalityDataset('processed_data/train.csv', 'processed_data/train_video', imagebind_model, "cuda")

In [ ]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import torch
from sklearn.metrics import f1_score
from IPython.display import clear_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming 'PersonalityDataset' is already defined and available
train_data = PersonalityDataset(
    'processed_data/train.csv',
    'processed_data/train_video',
    imagebind_model,
    device
)

data_loader = DataLoader(train_data, batch_size=1, shuffle=False)

In [ ]:
data_samples = []

for sample in data_loader:
    audio_embedding = sample['audio_embeddings'].squeeze(0).cpu().numpy()

    text_embedding = sample['transcript_embeddings'].squeeze(0).cpu().numpy()
    if text_embedding.ndim > 1:
        text_embedding = np.mean(text_embedding, axis=0)

    labels = sample['labels']
    label_values = {
        label_name: labels[label_name].item() if isinstance(labels[label_name], torch.Tensor) else labels[label_name]
        for label_name in [
            'extraversion',
            'neuroticism',
            'agreeableness',
            'conscientiousness',
            'openness',
            'interview'
        ]
    }

    data_samples.append({
        'audio_embedding': audio_embedding,
        'text_embedding': text_embedding,
        **label_values,
        'video_id': sample['video_file'][0]
    })
    clear_output(wait=True)

In [19]:
# Convert data_samples into a DataFrame
df = pd.DataFrame(data_samples)
# df.to_csv("embeddings.csv", index=False)
df.to_pickle("embeddings.pkl")

In [5]:
import torch
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import os
import tempfile
import subprocess
import librosa
from imagebind import model
from imagebind.model import ModalityType 
from imagebind.utils import data
from loguru import logger
import decord
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import torch
from sklearn.metrics import f1_score
from IPython.display import clear_output

df = pd.read_pickle("embeddings.pkl")

In [6]:
# Ensure the embedding columns are lists (if necessary)
df['audio_embedding'] = df['audio_embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
df['text_embedding'] = df['text_embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare the data for CatBoost
embedding_features = ['audio_embedding', 'text_embedding']
feature_columns = embedding_features  # Since we only have embeddings as features

X_train = train_df[feature_columns]
X_test = test_df[feature_columns]

y_train = train_df[['extraversion', 'neuroticism', 'agreeableness',
                    'conscientiousness', 'openness', 'interview']]
y_test = test_df[['extraversion', 'neuroticism', 'agreeableness',
                  'conscientiousness', 'openness', 'interview']]

label_names = ['extraversion', 'neuroticism', 'agreeableness',
               'conscientiousness', 'openness', 'interview']


In [18]:
regressors = {}

for label_name in label_names:
    print(f"Training model for label '{label_name}'")
    train_pool = Pool(
        data=X_train,
        label=y_train[label_name],
        embedding_features=embedding_features
    )
    test_pool = Pool(
        data=X_test,
        label=y_test[label_name],
        embedding_features=embedding_features
    )

    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.01,
        depth=6,
        loss_function='RMSE',
        eval_metric='RMSE',
        random_seed=42,
        verbose=100
    )

    model.fit(
        train_pool,
        eval_set=test_pool,
        use_best_model=True
    )

    regressors[label_name] = model

Training model for label 'extraversion'
0:	learn: 0.1521773	test: 0.1510782	best: 0.1510782 (0)	total: 1.74ms	remaining: 1.74s
100:	learn: 0.1357652	test: 0.1307827	best: 0.1307827 (100)	total: 93.9ms	remaining: 836ms
200:	learn: 0.1322818	test: 0.1262011	best: 0.1262011 (200)	total: 202ms	remaining: 804ms
300:	learn: 0.1310815	test: 0.1250740	best: 0.1250740 (300)	total: 291ms	remaining: 675ms
400:	learn: 0.1303153	test: 0.1247184	best: 0.1247152 (394)	total: 375ms	remaining: 560ms
500:	learn: 0.1296823	test: 0.1245797	best: 0.1245797 (500)	total: 461ms	remaining: 459ms
600:	learn: 0.1291033	test: 0.1245320	best: 0.1245319 (591)	total: 546ms	remaining: 362ms
700:	learn: 0.1285250	test: 0.1245245	best: 0.1245129 (647)	total: 633ms	remaining: 270ms
800:	learn: 0.1280001	test: 0.1245552	best: 0.1245129 (647)	total: 718ms	remaining: 178ms
900:	learn: 0.1274340	test: 0.1245735	best: 0.1245129 (647)	total: 805ms	remaining: 88.4ms
999:	learn: 0.1268635	test: 0.1246185	best: 0.1245129 (647)	t

In [24]:
predictions = {label_name: {} for label_name in label_names}
ground_truths = {label_name: {} for label_name in label_names}

for idx in train_df.index:
    x = train_df.loc[[idx], feature_columns]
    video_id = train_df.loc[idx, 'video_id']

    sample_pool = Pool(
        data=x,
        embedding_features=embedding_features
    )

    for label_name in label_names:
        model = regressors[label_name]
        y_pred = model.predict(sample_pool)[0]
        y_true = y_train.loc[idx, label_name]

        predictions[label_name][video_id] = y_pred
        ground_truths[label_name][video_id] = y_true

In [38]:
def calculate_tag_f1(pred: dict, truth: dict):
    epsilon = 0.1
    predicts = []
    truths = []
    for pred_key in list(pred.keys()):
        predict_value = pred[pred_key]
        if pred_key in list(truth.keys()):
            truth_value = truth[pred_key]
        else:
            raise Exception(f'Видеозапись с названием {pred_key} отсутствует в датасете!')
        predicts.append(1 if abs(predict_value - truth_value) < epsilon else 0)
        truths.append(1)

    while len(predicts) < len(list(truth.keys())):
        predicts.append(0)

    while len(truths) < len(list(truth.keys())):
        truths.append(1)

    return f1_score(truths, predicts, average='macro')


def calculate_f1(pred, truth):
    results = []

    extraversion_pred_raw = pred['extraversion']
    extraversion_truth_raw = truth['extraversion']
    results.append(calculate_tag_f1(extraversion_pred_raw, extraversion_truth_raw))

    neuroticism_pred_raw = pred['neuroticism']
    neuroticism_truth_raw = truth['neuroticism']
    results.append(calculate_tag_f1(neuroticism_pred_raw, neuroticism_truth_raw))

    agreeableness_pred_raw = pred['agreeableness']
    agreeableness_truth_raw = truth['agreeableness']
    results.append(calculate_tag_f1(agreeableness_pred_raw, agreeableness_truth_raw))

    conscientiousness_pred_raw = pred['conscientiousness']
    conscientiousness_truth_raw = truth['conscientiousness']
    results.append(calculate_tag_f1(conscientiousness_pred_raw, conscientiousness_truth_raw))

    openness_pred_raw = pred['openness']
    openness_truth_raw = truth['openness']
    results.append(calculate_tag_f1(openness_pred_raw, openness_truth_raw))

    return np.mean(results)

average_f1 = calculate_f1(predictions, ground_truths)
print(f"\nFinal Average F1 score across all labels: {average_f1:.4f}")


Final Average F1 score across all labels: 0.3933
